<a href="https://colab.research.google.com/github/sgoodfriend/rl-algo-impls/blob/main/rl_algo_impls/microrts/colab_microrts_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Python MicroRTS submission in Google Colab
[sgoodfriend/rl-algo-impls](https://github.com/sgoodfriend/rl-algo-impls) was used to [train a Python agent to play MicroRTS](https://wandb.ai/sgoodfriend/rl-algo-impls-benchmarks/reports/rl-algo-impls-MicroRTS-Training--Vmlldzo0NjA2NTAy). [Farama-Foundation/MicroRTS-Py](https://github.com/Farama-Foundation/MicroRTS-Py) was the inspiration for this training.

Given MicroRTS is a Java environment, a Python server runs the Pytorch model, which communicates over a socket connection. This Colab notebook demonstrates what is necessary to install the necessary Python dependencies and expose a command-line endpoint for the submission jar file to communicate with.

## Installation

1. Download the `rai.jar` submission file and `rl_algo_impls-0.0.16-py3-none-any.whl` Python installation file:

In [ ]:
%%capture
!apt-get install -y subversion
!svn export https://github.com/sgoodfriend/rl-algo-impls/tags/v0.0.16/rl_algo_impls/microrts/java/rai.jar
!wget https://github.com/sgoodfriend/rl-algo-impls/releases/download/v0.0.16/rl_algo_impls-0.0.16-py3-none-any.whl

2. Install dependencies and rl_algo_impls Python wheel. This exposes the `rai_microrts` command-line endpoint, which will be called by the jar file. Ubuntu 20.04's default JDK is Java 11. setuptools and wheel have to be downgraded for the installation because rl-algo-impls depends on stable-baslines3, which depends on gym 0.21.

In [ ]:
%%capture
# Output hidden for cleanliness; however, remove %%capture if any downstream issues
!apt install -y default-jdk
!pip install setuptools==65.5.0
!pip install wheel==0.38.4
!python -m pip install rl_algo_impls-0.0.16-py3-none-any.whl

3. Download the jar files (microrts.jar and bot jars) and maps to run the tournament:

In [ ]:
%%capture
# Output hidden for cleanliness; however, remove %%capture if any downstream issues
# microrts.jar and bot jars
!svn export https://github.com/sgoodfriend/rl-algo-impls/tags/v0.0.16/rl_algo_impls/microrts/java/lib
# Maps
!svn export https://github.com/sgoodfriend/rl-algo-impls/tags/v0.0.16/rl_algo_impls/microrts/java/maps

3. Run `tournaments.RAIRoundRobinTournament` which runs 10 iterations of the rai.jar `RAISocketAI` against 6 opponents on a 16x16 map. Depending on hardware, `RAISocketAI` can average from 50ms per turn on a Colab standard machine with 2-2.2GHz CPUs to 10ms per turn on modern CPUs, such as an Apple M1Max.

In [ ]:
!java -cp "rai.jar:$(find lib -name "*.jar" | tr '\n' ':')" tournaments.RAIRoundRobinTournament